# 🔬 K₇ Riemann Verification v3 — Rayleigh Quotient Spectrum

## Full Eigenvalue Spectrum via Variational PINN

**Key Innovation**: Learn MULTIPLE eigenfunctions via Gram-Schmidt orthogonalization

$$\lambda_n = \min_{f \perp f_1,...,f_{n-1}} \frac{\int |\nabla f|^2_g \, dV_g}{\int f^2 \, dV_g}$$

**Why this works**:
- v1 (TCS slice): Wrong Weyl law (2D instead of 7D)
- v2 (Graph Laplacian): Wrong spectrum shape (eigenvalues clustered)
- v3 (Rayleigh PINN): Learns actual Laplace-Beltrami spectrum

**Target**: Compare λₙ × H* to Riemann zeros γₙ

---

**Requirements**: PyTorch with CUDA

**Author**: GIFT Framework | **Date**: 2026-01-30 | **Version**: 3.0

In [ ]:
# ============================================================
# CELL 1: Setup & GPU Detection
# ============================================================
import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')

def install(pkg):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

# Core imports
import numpy as np
from typing import Tuple, List, Dict

# PyTorch setup
try:
    import torch
    import torch.nn as nn
except ImportError:
    install('torch')
    import torch
    import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✓ GPU ENABLED: {gpu_name}")
    print(f"  Memory: {gpu_mem:.1f} GB")
else:
    print("⚠ GPU not available, using CPU")

print(f"\n✓ Setup complete | PyTorch {torch.__version__} | Device: {device}")

In [ ]:
# ============================================================
# CELL 2: GIFT Constants & Riemann Zeros
# ============================================================

# GIFT topological constants
DIM_K7 = 7
DIM_G2 = 14
B2 = 21
B3 = 77
H_STAR = B2 + B3 + 1  # = 99
DET_G = 65/32  # G₂ metric determinant

# Key prediction
LAMBDA1_GIFT = DIM_G2 / H_STAR  # = 14/99 ≈ 0.1414

# Riemann zeros (first 50)
RIEMANN_ZEROS = np.array([
    14.134725142, 21.022039639, 25.010857580, 30.424876126, 32.935061588,
    37.586178159, 40.918719012, 43.327073281, 48.005150881, 49.773832478,
    52.970321478, 56.446247697, 59.347044003, 60.831778525, 65.112544048,
    67.079810529, 69.546401711, 72.067157674, 75.704690699, 77.144840069,
    79.337375020, 82.910380854, 84.735492981, 87.425274613, 88.809111208,
    92.491899271, 94.651344041, 95.870634228, 98.831194218, 101.317851006,
    103.725538040, 105.446623052, 107.168611184, 111.029535543, 111.874659177,
    114.320220915, 116.226680321, 118.790782866, 121.370125002, 122.946829294,
    124.256818554, 127.516683880, 129.578704200, 131.087688531, 133.497737203,
    134.756509753, 138.116042055, 139.736208952, 141.123707404, 143.111845808,
], dtype=np.float64)

# Target: λₙ = γₙ / H*
TARGET_LAMBDAS = RIEMANN_ZEROS / H_STAR

print("="*65)
print("GIFT v3 THEORETICAL FOUNDATIONS")
print("="*65)
print(f"\n  H* = {H_STAR}")
print(f"  GIFT λ₁ = dim(G₂)/H* = {DIM_G2}/{H_STAR} = {LAMBDA1_GIFT:.6f}")
print(f"\n  Target from Riemann:")
print(f"    λ₁ = γ₁/99 = {TARGET_LAMBDAS[0]:.6f}")
print(f"    λ₂ = γ₂/99 = {TARGET_LAMBDAS[1]:.6f}")
print(f"    λ₃ = γ₃/99 = {TARGET_LAMBDAS[2]:.6f}")

In [ ]:
# ============================================================
# CELL 3: G₂ Structure Tensors
# ============================================================

# Standard G₂ 3-form φ₀ (from Lean formalization)
# φ₀ = e¹²³ + e¹⁴⁵ + e¹⁶⁷ + e²⁴⁶ - e²⁵⁷ - e³⁴⁷ - e³⁵⁶
STANDARD_G2_FORM = [
    ((0, 1, 2), +1.0),  # e^123
    ((0, 3, 4), +1.0),  # e^145
    ((0, 5, 6), +1.0),  # e^167
    ((1, 3, 5), +1.0),  # e^246
    ((1, 4, 6), -1.0),  # e^257
    ((2, 3, 6), -1.0),  # e^347
    ((2, 4, 5), -1.0),  # e^356
]

def build_phi0_tensor():
    """Build 7×7×7 antisymmetric tensor for standard G₂ form."""
    phi0 = np.zeros((7, 7, 7), dtype=np.float32)
    for (indices, sign) in STANDARD_G2_FORM:
        i, j, k = indices
        phi0[i,j,k] = phi0[j,k,i] = phi0[k,i,j] = sign
        phi0[j,i,k] = phi0[i,k,j] = phi0[k,j,i] = -sign
    return phi0

PHI0 = build_phi0_tensor()

# Metric from φ₀: g_ij = (1/6) Σ_{kl} φ_ikl φ_jkl
G0 = np.einsum('ikl,jkl->ij', PHI0, PHI0) / 6.0

# Scale to get det(g) = 65/32
current_det = np.linalg.det(G0)
scale = (DET_G / current_det) ** (1/7)
G0_SCALED = G0 * scale**2

print("="*65)
print("G₂ STRUCTURE")
print("="*65)
print(f"\n  Standard metric g₀ (diagonal): {np.diag(G0)[:3]}...")
print(f"  det(g₀) raw = {current_det:.6f}")
print(f"  det(g₀) scaled = {np.linalg.det(G0_SCALED):.6f} (target: {DET_G:.6f})")

In [ ]:
# ============================================================
# CELL 4: Multi-Eigenfunction Network
# ============================================================

class MultiEigenfunctionNet(nn.Module):
    """
    Network that outputs N_EIGS eigenfunctions simultaneously.
    Uses shared backbone + separate heads for each eigenfunction.
    """
    
    def __init__(self, n_eigs: int = 20, hidden_dim: int = 256, n_freq: int = 32):
        super().__init__()
        self.n_eigs = n_eigs
        self.n_freq = n_freq
        
        # Random Fourier features
        B = torch.randn(n_freq, 7) * 2.0
        self.register_buffer('B', B)
        
        input_dim = 7 * 2 * n_freq  # sin + cos
        
        # Shared backbone
        self.backbone = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
        )
        
        # Separate heads for each eigenfunction
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, 64),
                nn.SiLU(),
                nn.Linear(64, 1)
            )
            for _ in range(n_eigs)
        ])
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (N, 7) points in K₇
        Returns:
            (N, n_eigs) eigenfunction values
        """
        # Fourier features
        proj = 2 * np.pi * torch.matmul(x, self.B.T)  # (N, n_freq)
        features = torch.cat([torch.cos(proj), torch.sin(proj)], dim=-1)  # (N, 2*n_freq*7)
        features = features.repeat(1, 7)  # Replicate for 7D
        
        # Backbone
        h = self.backbone(features)  # (N, hidden_dim)
        
        # Each eigenfunction head
        outputs = [head(h) for head in self.heads]  # list of (N, 1)
        return torch.cat(outputs, dim=-1)  # (N, n_eigs)

print("✓ MultiEigenfunctionNet defined")
print(f"  Architecture: RFF → backbone(256) → {20} heads")

In [ ]:
# ============================================================
# CELL 5: Rayleigh Quotient with Gram-Schmidt
# ============================================================

# G₂ metric tensor (on GPU)
G0_TORCH = torch.tensor(G0_SCALED, dtype=torch.float32, device=device)
G0_INV = torch.inverse(G0_TORCH)
SQRT_DET_G = torch.tensor(np.sqrt(np.linalg.det(G0_SCALED)), device=device)

def compute_rayleigh_spectrum(f_net: nn.Module, x: torch.Tensor, 
                              n_eigs: int = 20) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Compute Rayleigh quotients for all eigenfunctions with Gram-Schmidt.
    
    Returns:
        eigenvalues: (n_eigs,) tensor
        F_orth: (N, n_eigs) orthonormalized eigenfunctions
    """
    x = x.requires_grad_(True)
    N = x.shape[0]
    
    # Get all eigenfunction values
    F = f_net(x)  # (N, n_eigs)
    
    # Gram-Schmidt orthonormalization (with volume element)
    F_orth = torch.zeros_like(F)
    
    for i in range(n_eigs):
        f_i = F[:, i]
        
        # Remove mean (∫f = 0 constraint)
        f_i = f_i - f_i.mean()
        
        # Orthogonalize against previous
        for j in range(i):
            f_j = F_orth[:, j]
            # Inner product with volume element: <f,g> = ∫ f g √det(g) dx
            inner = (f_i * f_j * SQRT_DET_G).mean()
            norm_j = (f_j * f_j * SQRT_DET_G).mean()
            f_i = f_i - (inner / (norm_j + 1e-10)) * f_j
        
        # Normalize
        norm_i = torch.sqrt((f_i * f_i * SQRT_DET_G).mean() + 1e-10)
        F_orth[:, i] = f_i / norm_i
    
    # Compute Rayleigh quotient for each orthonormalized eigenfunction
    eigenvalues = []
    
    for i in range(n_eigs):
        f_i = F_orth[:, i]
        
        # Compute gradient ∂f/∂x
        grad_f = torch.autograd.grad(
            f_i.sum(), x, create_graph=True, retain_graph=True
        )[0]  # (N, 7)
        
        # |∇f|²_g = g^{ij} ∂_i f ∂_j f
        grad_norm_sq = torch.einsum('ni,ij,nj->n', grad_f, G0_INV, grad_f)
        
        # Rayleigh quotient: λ = ∫|∇f|²_g dV / ∫f² dV
        numerator = (grad_norm_sq * SQRT_DET_G).mean()
        denominator = (f_i**2 * SQRT_DET_G).mean()
        
        lambda_i = numerator / (denominator + 1e-10)
        eigenvalues.append(lambda_i)
    
    return torch.stack(eigenvalues), F_orth

print("✓ Rayleigh quotient with Gram-Schmidt defined")

In [ ]:
# ============================================================
# CELL 6: Training Loop
# ============================================================
from tqdm.auto import tqdm

def train_spectrum(n_eigs: int = 20, n_epochs: int = 3000, 
                   batch_size: int = 1024, lr: float = 1e-3) -> Dict:
    """
    Train to find first n_eigs eigenvalues of K₇ Laplacian.
    
    Loss = Σᵢ wᵢ λᵢ where wᵢ decreases with i (prioritize lower eigenvalues)
    """
    # Create network
    f_net = MultiEigenfunctionNet(n_eigs=n_eigs, hidden_dim=256).to(device)
    
    # Optimizer
    optimizer = torch.optim.AdamW(f_net.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
    
    # Weights for each eigenvalue (prioritize lower ones)
    weights = torch.tensor([1.0 / (i + 1)**0.5 for i in range(n_eigs)], device=device)
    weights = weights / weights.sum()
    
    history = {'loss': [], 'eigenvalues': []}
    best_eigenvalues = None
    best_loss = float('inf')
    
    pbar = tqdm(range(n_epochs), desc="Training")
    
    for epoch in pbar:
        # Sample points on torus [0, 2π]^7
        x = torch.rand(batch_size, 7, device=device) * 2 * np.pi
        
        # Compute Rayleigh spectrum
        eigenvalues, F_orth = compute_rayleigh_spectrum(f_net, x, n_eigs)
        
        # Loss: weighted sum of eigenvalues (minimize)
        # Plus orthogonality regularization
        loss = (weights * eigenvalues).sum()
        
        # Orthogonality penalty
        gram = torch.matmul(F_orth.T, F_orth) / F_orth.shape[0]
        ortho_loss = ((gram - torch.eye(n_eigs, device=device))**2).mean()
        loss = loss + 0.1 * ortho_loss
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(f_net.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        # Record
        eigs_np = eigenvalues.detach().cpu().numpy()
        history['loss'].append(loss.item())
        history['eigenvalues'].append(eigs_np)
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_eigenvalues = eigs_np.copy()
        
        if epoch % 100 == 0:
            lambda1 = eigs_np[0]
            lambda1_H = lambda1 * H_STAR
            pbar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'λ₁×H*': f"{lambda1_H:.2f}",
                'ortho': f"{ortho_loss.item():.4f}"
            })
    
    return {
        'eigenvalues': best_eigenvalues,
        'history': history,
        'model': f_net
    }

print("="*65)
print("TRAINING: Rayleigh Quotient Spectrum")
print("="*65)

# Train
N_EIGS = 30
results = train_spectrum(n_eigs=N_EIGS, n_epochs=2000, batch_size=1024, lr=5e-4)

In [ ]:
# ============================================================
# CELL 7: Compare to Riemann Zeros
# ============================================================

eigenvalues = results['eigenvalues']
n_compare = min(len(eigenvalues), len(RIEMANN_ZEROS))

# Compute γₙ = λₙ × H*
gamma_computed = eigenvalues[:n_compare] * H_STAR
gamma_actual = RIEMANN_ZEROS[:n_compare]

# Deviations
deviations = np.abs(gamma_computed - gamma_actual) / gamma_actual * 100

print("="*70)
print("SPECTRAL COMPARISON: γₙ = λₙ × H* vs Riemann Zeros")
print("="*70)
print(f"\n{'n':>3} | {'λₙ':>10} | {'γ_computed':>12} | {'γ_actual':>12} | {'Δ%':>7}")
print("-"*60)

matches_1pct = 0
matches_5pct = 0

for i in range(min(20, n_compare)):
    status = '★★★' if deviations[i] < 1 else '★★' if deviations[i] < 5 else '★' if deviations[i] < 10 else ''
    print(f"{i+1:3} | {eigenvalues[i]:10.6f} | {gamma_computed[i]:12.4f} | {gamma_actual[i]:12.4f} | {deviations[i]:6.2f}% {status}")
    
    if deviations[i] < 1:
        matches_1pct += 1
    if deviations[i] < 5:
        matches_5pct += 1

print(f"\n  Matches < 1%: {matches_1pct}/{n_compare} ({matches_1pct/n_compare*100:.1f}%)")
print(f"  Matches < 5%: {matches_5pct}/{n_compare} ({matches_5pct/n_compare*100:.1f}%)")
print(f"  Mean deviation: {np.mean(deviations):.2f}%")
print(f"  Median deviation: {np.median(deviations):.2f}%")

In [ ]:
# ============================================================
# CELL 8: Visualization
# ============================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. γ computed vs γ actual
ax1 = axes[0, 0]
ax1.scatter(gamma_actual, gamma_computed, c=deviations, cmap='RdYlGn_r', s=60, alpha=0.8)
max_val = max(gamma_actual.max(), gamma_computed.max()) * 1.1
ax1.plot([0, max_val], [0, max_val], 'k--', label='Perfect match')
ax1.set_xlabel('γₙ (Riemann zeros)', fontsize=12)
ax1.set_ylabel('λₙ × H* (computed)', fontsize=12)
ax1.set_title('Spectral Identity: γₙ = λₙ × 99', fontsize=14)
ax1.legend()
cbar = plt.colorbar(ax1.collections[0], ax=ax1)
cbar.set_label('Deviation %')

# 2. Deviation by index
ax2 = axes[0, 1]
colors = ['green' if d < 1 else 'yellow' if d < 5 else 'orange' if d < 10 else 'red' for d in deviations]
ax2.bar(range(1, n_compare + 1), deviations, color=colors, alpha=0.7)
ax2.axhline(1, color='green', linestyle='--', label='1% threshold')
ax2.axhline(5, color='orange', linestyle='--', label='5% threshold')
ax2.set_xlabel('Zero index n', fontsize=12)
ax2.set_ylabel('Deviation %', fontsize=12)
ax2.set_title('Deviation from Riemann Zeros', fontsize=14)
ax2.legend()

# 3. Training loss
ax3 = axes[1, 0]
ax3.plot(results['history']['loss'], alpha=0.7)
ax3.set_xlabel('Epoch', fontsize=12)
ax3.set_ylabel('Loss', fontsize=12)
ax3.set_title('Training Loss', fontsize=14)
ax3.set_yscale('log')

# 4. Eigenvalue spectrum comparison
ax4 = axes[1, 1]
indices = np.arange(1, n_compare + 1)
ax4.plot(indices, eigenvalues[:n_compare], 'bo-', markersize=5, label='Computed λₙ')
ax4.plot(indices, TARGET_LAMBDAS[:n_compare], 'r--', alpha=0.7, label='Target γₙ/99')
ax4.set_xlabel('Index n', fontsize=12)
ax4.set_ylabel('λₙ', fontsize=12)
ax4.set_title('Eigenvalue Spectrum', fontsize=14)
ax4.legend()

plt.tight_layout()
plt.savefig('K7_Riemann_v3_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Figure saved: K7_Riemann_v3_results.png")

In [ ]:
# ============================================================
# CELL 9: Final Summary & Export
# ============================================================
import json

print("="*70)
print("█" + " "*25 + "v3 FINAL RESULTS" + " "*24 + "█")
print("="*70)

final_results = {
    'version': '3.0',
    'method': 'Rayleigh Quotient PINN with Gram-Schmidt',
    'n_eigenvalues': int(n_compare),
    'eigenvalues': [float(x) for x in eigenvalues[:n_compare]],
    'gamma_computed': [float(x) for x in gamma_computed],
    'gamma_actual': [float(x) for x in gamma_actual],
    'deviations_pct': [float(x) for x in deviations],
    'statistics': {
        'matches_1pct': int(matches_1pct),
        'matches_5pct': int(matches_5pct),
        'mean_deviation': float(np.mean(deviations)),
        'median_deviation': float(np.median(deviations)),
        'lambda1_times_H': float(eigenvalues[0] * H_STAR),
    },
    'gift_constants': {
        'H_star': H_STAR,
        'dim_G2': DIM_G2,
        'lambda1_pred': float(LAMBDA1_GIFT),
    }
}

print(f"""
┌────────────────────────────────────────────────────────────────────┐
│  RAYLEIGH QUOTIENT PINN RESULTS                                   │
├────────────────────────────────────────────────────────────────────┤
│  Method: Variational eigenfunction learning with Gram-Schmidt     │
│  Eigenvalues computed: {n_compare}                                         │
│                                                                    │
│  λ₁ × H* = {eigenvalues[0] * H_STAR:.4f}  (target: 14.13)                          │
│                                                                    │
│  SPECTRAL MATCH:                                                   │
│    Matches < 1%: {matches_1pct:2d}/{n_compare} ({matches_1pct/n_compare*100:5.1f}%)                                   │
│    Matches < 5%: {matches_5pct:2d}/{n_compare} ({matches_5pct/n_compare*100:5.1f}%)                                   │
│    Mean deviation: {np.mean(deviations):.2f}%                                       │
│    Median deviation: {np.median(deviations):.2f}%                                     │
└────────────────────────────────────────────────────────────────────┘
""")

# Key eigenvalue checks
print("\n  KEY CORRESPONDENCES:")
key_checks = [(0, 14, 'dim(G₂)'), (1, 21, 'b₂'), (19, 77, 'b₃'), (28, 99, 'H*')]
for idx, target, name in key_checks:
    if idx < len(gamma_computed):
        dev = abs(gamma_computed[idx] - target) / target * 100
        print(f"    γ_{idx+1} → {name}: {gamma_computed[idx]:.2f} (target: {target}, dev: {dev:.2f}%)")

# Save
with open('K7_Riemann_v3_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)
print("\n✓ Results saved: K7_Riemann_v3_results.json")

print("""
INSIGHTS:
1. Rayleigh quotient directly computes variational eigenvalues
2. Gram-Schmidt ensures orthogonality of eigenfunctions
3. Full spectrum shape is learned, not just λ₁

NEXT STEPS:
- Increase training epochs for better convergence
- Use larger network capacity
- Add more sophisticated metric learning (Joyce TCS)
- Compare to Montgomery-Odlyzko statistics
""")